In [1]:
import pandas as pd
import csv
import numpy as np
import sys
from operator import itemgetter
from collections import Counter
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import preprocessing
import random
import math

In [2]:
# read the required file 
data= pd.read_csv("/Users/SKU/Desktop/fall2017/data mining/Project 3/project3_dataset1.txt", sep="\t", header=None)
print(data)

         0      1       2       3        4        5         6         7   \
0    20.130  28.25  131.20  1261.0  0.09780  0.10340  0.144000  0.097910   
1    12.450  15.70   82.57   477.1  0.12780  0.17000  0.157800  0.080890   
2    11.260  19.96   73.72   394.1  0.08020  0.11810  0.092740  0.055880   
3    11.430  15.39   73.06   399.8  0.09639  0.06889  0.035030  0.028750   
4    14.610  15.69   92.68   664.9  0.07618  0.03515  0.014470  0.018770   
5    15.340  14.26  102.50   704.4  0.10730  0.21350  0.207700  0.097560   
6    11.890  17.36   76.20   435.6  0.12250  0.07210  0.059290  0.074040   
7    13.750  23.77   88.54   590.0  0.08043  0.06807  0.046970  0.023440   
8    15.080  25.74   98.00   716.6  0.10240  0.09769  0.123500  0.065530   
9    19.270  26.47  127.90  1162.0  0.09401  0.17190  0.165700  0.075930   
10   19.550  23.21  128.90  1174.0  0.10100  0.13180  0.185600  0.102100   
11    9.668  18.10   61.06   286.3  0.08311  0.05428  0.014790  0.005769   
12   11.360 

In [3]:
# Separate Attribute and Label
labels = data[data.columns[-1]]
data = data.ix[:, :len(data.columns)-2]
#print(data.head())

/Users/SKU/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Handeling Categorical Data

#Select Categorical Columns from the dataframe
obj_df = data.select_dtypes(include=['object']).copy()

#One-Hot Encoding for categorical data
for idx in obj_df.columns:
    obj_df= pd.get_dummies(obj_df,columns=[idx],drop_first=False)

obj_dataset = obj_df.values.tolist()
#print(obj_dataset)

In [5]:
# Handeling Numerical Data
#Select numerical data from the data
num_df = data.select_dtypes(include=['int64','float64']).copy()

# Normalize Numeric Data
if (len(num_df.columns) !=0):
    num_dataset = preprocessing.normalize(num_df, norm='l2')
    num_dataset
else:
    num_dataset = []


In [6]:
# Dataset Format to be used for algorithm
if len(num_dataset)!= 0:
    dataset = np.column_stack((num_dataset, obj_dataset))
    dataset = np.column_stack((dataset, labels))
else:
    dataset = np.column_stack((obj_dataset, labels))
(len(dataset[0]))

31

In [7]:
last_col = len(dataset[0])-1
print(last_col)

30


In [8]:
# #  STANDARDIZE THE CONTINUOUS COLUMNS

# # To standardize using min-max
# minimum = np.min(temp_train, axis=0)
# maximum = np.max(temp_train, axis=0)


# minimumTest = np.min(temp_test, axis=0)
# maximumTest = np.max(temp_test, axis=0)

# # To standardize using Z-Score

# mean = np.mean(temp_train, axis=0)
# var = np.var(temp_train,axis=0)

In [9]:

# #STANDARDISE THE DATA
# for i in range(0,temp_train.shape[1]):
#     for j in range(0,temp_train.shape[0]):
# #         temp_train[j][i] = (temp_train[j,i] - minimum[i])/(maximum[i] - minimum[i])
#         temp_train[j][i] = (temp_train[j][i] - mean[i])/ var[i]

# for i in range(0,temp_test.shape[1]):
#     for j in range(0,temp_test.shape[0]):
# #         temp_test[j][i] = (temp_test[j,i] - minimumTest[i])/(maximumTest[i] - minimumTest[i])
#         temp_test[j][i] = (temp_test[j][i] - mean[i])/ var[i]
    
    
    

In [10]:
def standardize_datasets(temp_train, temp_test):
    
    # first block of code

    # To standardize using min-max
    minimum = np.min(temp_train, axis=0)
    maximum = np.max(temp_train, axis=0)


    minimumTest = np.min(temp_test, axis=0)
    maximumTest = np.max(temp_test, axis=0)

    # To standardize using Z-Score

    mean = np.mean(temp_train, axis=0)
    var = np.var(temp_train,axis=0)
    
    
    #STANDARDISE THE DATA
    for i in range(0,temp_train.shape[1]):
        for j in range(0,temp_train.shape[0]):
            temp_train[j][i] = (temp_train[j][i] - mean[i])/ var[i]

    for i in range(0,temp_test.shape[1]):
        for j in range(0,temp_test.shape[0]):
            temp_test[j][i] = (temp_test[j][i] - mean[i])/ var[i]
    
    return temp_test , temp_train

In [11]:
# divide entire data into n folds
def cross_validation(n_folds):
    updated_datasets = list(dataset)
    size_fold = math.floor(len(updated_datasets)/n_folds)
    new_dataset = []
    for i in range(0, n_folds):
        count = 0
        fold = []
        while count < size_fold:
            r = random.randint(0, len(updated_datasets)-1)
            fold.append(updated_datasets[r])
            updated_datasets.pop(r)
            count +=1
        new_dataset.append(fold)
    return new_dataset

In [12]:
def get_foldset(updatedset, test_index): 
    foldset = []
    
    #remove the test set and combine other lists for train 
    newset = list(updatedset)
    newset.pop(test_index)
    foldset = sum(newset, [])
 
    return foldset

In [13]:
random.seed(1234)
n_fold = 10
validated_dataset = cross_validation(n_fold)
accuracies = []
precisions = []
recalls = []
f1_measures = []
k_val = 9

for i in range(0, n_fold):
    print("----------------beginnning of a fold---------------------")
    test_data = np.asarray(validated_dataset[i])
    train_data = np.asarray(get_foldset(validated_dataset, i))
    
    correct_predicted = 0
    true_positive = 0
    false_positive = 0
    f_n = 0
    
    #delete last column from the datasets
    temp_train = np.delete(train_data, last_col, 1)
    temp_test = np.delete(test_data, last_col, 1)
    
    for i in range(0, temp_test.shape[0]):

        eu_distance = []
        knn = []
    
        for j in range(0, temp_train.shape[0]):
        
            eu_dist = np.linalg.norm(temp_test[i] - temp_train[j])
            eu_distance.append([j , eu_dist])
    
        eu_distance = sorted(eu_distance, key=itemgetter(1))
    
        knn = eu_distance[:k_val]
    
        class0 = 0
        class1 = 0
        for k in knn:
            classValue = train_data[int(k[0])][last_col]
            if classValue == 0.0:
                class0 = class0 + 1
            else:
                class1 = class1 + 1
                
        if class0 > class1:
            result = 0.0
        else:
            result = 1.0
    
    
        expected = test_data[i][last_col]
   
        if expected == result:
            correct_predicted += 1
            #print(correct_predicted)
        
        if (expected == 1 and result == 1):
            true_positive +=1
        
        if expected == 0 and result == 1:
            false_positive +=1
        
        if expected == 1 and result == 0:
            f_n += 1
     
    # calculate measure for each fold
    Accuracy = float(correct_predicted) / len(test_data)
    if true_positive != 0:
        Precision = float(true_positive)/ (true_positive + false_positive)
        Recall = float(true_positive)/ (true_positive + f_n)
        f1_measure = float((2 * true_positive ))/ ((2 * true_positive ) + f_n + false_positive )
    else:
        Precision = 0
        Recall = 0
        f1_measure = 0
    
    #append result in the list for all the folds
    accuracies.append(Accuracy)
    precisions.append(Precision)
    recalls.append(Recall)
    f1_measures.append(f1_measure)
    

resultant_accuracy = sum(accuracies)/ len(accuracies)
resultant_precision = sum(precisions)/ len(precisions)
resultant_recall = sum(recalls)/ len(recalls)
resultant_f1measure = sum(f1_measures)/len(f1_measures)

print("------------------------------------------------")
print("Final accuracy::" + str(resultant_accuracy))
print("Final precision::" + str(resultant_precision))
print("Final recall::" + str(resultant_recall))
print("Final f1::" + str(resultant_f1measure))



----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
----------------beginnning of a fold---------------------
------------------------------------------------
Final accuracy::0.9196428571428571
Final precision::0.9359031392584025
Final recall::0.8406291866028708
Final f1::0.8842509157638412
